In [99]:
from matplotlib import pyplot as plt


import os, fitz, cv2
import numpy as np

def determine_vertical_line(pix, index):
	image_bytes = pix.samples
	image = np.frombuffer(image_bytes, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
	if pix.n == 4:
		image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)
	gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	ret, binary_image = cv2.threshold(gray_img, 240, 255, cv2.THRESH_BINARY)
	height, width = binary_image.shape
	center = width // 2
	thickness = 20 # 검사할 선의 두께
	center_line = binary_image[100:height-100, center-thickness//2:center+thickness//2]
	
	kernel = np.ones((5, 5), np.uint8)
	opening = cv2.morphologyEx(center_line, cv2.MORPH_OPEN, kernel)
	
	for r in range(20, opening.shape[0], 20):
		piece = opening[r-20:r, :]
		y_coords, x_coords = np.where(piece == 0)

		if y_coords.size == 0 or np.max(piece) == 0:
			return

		prev_y, prev_x = y_coords[0], x_coords[0]
		for y in y_coords:
			if abs(y-prev_y) >= 5:
				return
			prev_y = y

	# plt.title(f"{index}")
	# plt.imshow(opening, cmap='gray_r')
	# plt.show()
	return True

	
def convert_pdf_to_png(pdf_file, output_folder, pdf_page_number = 0):
	pure_file_name = os.path.basename(pdf_file)[:-4].replace(" ", "")

	if not os.path.exists(output_folder):
		os.mkdir(output_folder)
	
	doc = fitz.open(pdf_file)

	
	try:
		if pdf_page_number == 0: # pdf_page_number 특정 값 미지정 시, 전체 변환
			for i, page in enumerate(doc):
				img = page.get_pixmap()   # 이미지 변환
				if determine_vertical_line(pix=img, index=i+1):
					print(i+1)
					img.save(output_folder + "\\" + pure_file_name + f'_{i}th_problem_page.png') # 변환된 이미지 저장
				
			print('전체 변환')
		elif pdf_page_number != 0:
			page = doc.load_page(pdf_page_number - 1) # 특정 페이지 가져오기
			i = pdf_page_number
			img = page.get_pixmap()   # 이미지 변환
			img.save(output_folder + "\\" + pure_file_name + f'_{i}_only_output.png') # 변환된 이미지 저장
			
			print(pdf_page_number, '페이지 변환')

		
	except ValueError:
		print('Error: page not in document')

fileName = "[블랙라벨] 수학 II.pdf"
s3_textbook_path = f"textbook/2024/{fileName}"

globalUtils_dir = r"C:\Users\aiotu\Projects\GradProj\cheada_fastapi\cheada\globalUtils"
local_textbook_dir = r"C:\Users\aiotu\Projects\GradProj\books"
temp_page_storage = f"{globalUtils_dir}\\temp_page_storage\\{fileName[:-4]}"
temp_problem_storage = f"{globalUtils_dir}\\temp_problem_storage"

convert_pdf_to_png(pdf_file="C:\\Users\\aiotu\\Projects\\GradProj\\books\\[블랙라벨] 수학 II.pdf", output_folder=temp_page_storage)
	

13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
33
34
35
36
37
38
39
40
41
43
44
45
46
47
48
49
50
51
53
54
55
56
57
58
59
60
61
65
66
67
68
69
70
71
72
73
75
76
77
78
79
80
81
82
전체 변환
